In [1]:
from typing import List, OrderedDict, Counter, Optional, Tuple
from collections import defaultdict
import collections
import operator

### Graph Sum

- Given a maze with N cells. Each cell may have multiple entry points but not more than one exit(i.e entry/exit points are unidirectional doors like valves).

- You are given an array Edge[] of N integers, where Edge[i] contains the cell number that can be reached from of cell i in one step. Edge[i] is -1 if the ith cell doesn’t have an exit.

#### Max cycle sum

In [4]:
import math
class Solution:
    def getMaxCycleSum(self, numberOfNodes: int, connections: List[int]) -> int:
        graph = defaultdict(list)
        
        for u, v in enumerate(connections):
            if v == -1: continue
            graph[u].append(v)

        seen = set()
        curr_path = [-1] * numberOfNodes
        
        def dfs(node, curr_sum):
            nonlocal seen, curr_path
            seen.add(node)
            curr_path[node] = curr_sum
            
            max_sum = -math.inf

            for nei in graph[node]:
                if nei not in seen:
                    max_sum = max(max_sum, dfs(nei, curr_sum + nei))
                elif curr_path[nei] != -1:
                    # total - cost to reach nei + including nei
                    max_sum = max(max_sum, curr_sum - curr_path[nei] + nei)

            curr_path[node] = -1
            return max_sum
        
        res = 0
        
        for node in range(numberOfNodes):
            if node not in seen:
                cycle_sum = dfs(node, node)
                res = max(res, cycle_sum)

        return res
            
sol = Solution()
print(sol.getMaxCycleSum(23, [4, 4, 1, 4, 13, 8, 8, 8, 0, 8, 14, 9, 15, 11, -1, 10, 15, 22, 22, 22, 22, 22, 21])) #45

45


#### Node number of maximum weighted node

In [10]:
class Solution:
    def getMaxWeightedNode(self, numOfNodes: int, connections: List[int]) -> int:
        in_map = [0] * (numOfNodes + 1) # sum of all incoming nodes

        res, curr_mx = -1, -1
        for u, v in enumerate(connections): 
            if v == -1: continue
            in_map[v] += u
            
            if in_map[v] > curr_mx:
                curr_mx = in_map[v]
                res = v
            
        return res


sol = Solution()
print(sol.getMaxWeightedNode(23, [4, 4, 1, 4, 13, 8, 8, 8, 0, 8, 14, 9, 15, 11, -1, 10, 15, 22, 22, 22, 22, 22, 21])) #22

22


#### Nearest Meeting cell

In [23]:
class Solution:
    def getNearestMeetingCell(self, numOfNodes: int, connections: List[int], a: int, b: int) -> int:
        x, y = a, b
        a_path, b_path = set(), set()
        
        while True:
            # cycle condition or both have reached an end.
            if x in a_path or y in b_path or (x == -1 and y == -1):
                return -1
            
            # if current node is already encountered in other node's path
            if x in b_path:
                return x
            
            # if current node is already encountered in other node's path
            if y in a_path:
                return y

            # if the node hasn't reached the end move by one step 
            if x != -1:
                a_path.add(x)
                x = connections[x]

            # if the node hasn't reached the end move by one step
            if y != -1:
                b_path.add(y)
                y = connections[y]
        

sol = Solution()
print(sol.getNearestMeetingCell(23, [4, 4, 1, 4, 13, 8, 8, 8, 0, 8, 14, 9, 15, 11, -1, 10, 15, 22, 22, 22, 22, 22, 21], 2, 9)) #4
print(sol.getNearestMeetingCell(23, [4, 4, 1, 4, 13, 8, 8, 8, 0, 8, 14, 9, 15, 11, -1, 10, 15, 22, 22, 22, 22, 22, 21], 12, 14)) #14
print(sol.getNearestMeetingCell(23, [4, 4, 1, 4, 13, 8, 8, 8, 0, 8, 14, 9, 15, 11, -1, 10, 15, 22, 22, 22, 22, 22, 21], 2, 13)) #13
print(sol.getNearestMeetingCell(23, [4, 4, 1, 4, 13, 8, 8, 8, 0, 8, 14, 9, 15, 11, -1, 10, 15, 22, 22, 22, 22, 22, 21], 2, 11)) #11
print(sol.getNearestMeetingCell(23, [4, 4, 1, 4, 13, 8, 8, 8, 0, 8, 14, 9, 15, 11, -1, 10, 15, 22, 22, 22, 22, 22, 21], 2, 14)) #11

4
14
13
11
-1


### Meetup Schedule

1             |  2          | 3
:-:|:-:|:-:
![alt text](Visa_img/meetup_1.png "1")  |  ![alt text](Visa_img/meetup_2.png "1") | ![alt text](Visa_img/meetup_3.png "1")



#### Count number of meetings

In [68]:
import heapq
class Solution:
    def countMeetings_wrong(self, arrival: List[int], departure: List[int]) -> int:
        investor_tup = []
        
        for i in range(len(arrival)):
            investor_tup.append((arrival[i], departure[i], i))

        investor_tup.sort(key = lambda x: (x[0], x[1]))
        print(investor_tup)
        last_day = investor_tup[len(arrival) - 1][1]
        
        res, i = 0, 0
        done = set()
        for day in range(1, last_day + 1):
            while True:
                if i == len(arrival): return res

                start, end = investor_tup[i][0], investor_tup[i][1]
                if start <= day <= end: break
                i += 1
            
            res += 1
            done.add(investor_tup[i][2])
            i += 1
        print(done)
        return res
    
    def countMeetings(self, arrival: List[int], departure: List[int]) -> int:
        start_map = defaultdict(list)
        i, j = float('inf'), 0

        for s, e in zip(arrival, departure):
            start_map[s].append(e)
            i = min(i, s)
            j = max(j, e)
        
        start_heap = []
        res = 0
        for s in range(i, j + 1):
            while start_heap and start_heap[0] < s:
                heapq.heappop(start_heap)

            for e in start_map[s]:
                heapq.heappush(start_heap, e)

            if start_heap:
                res += 1
                heapq.heappop(start_heap)
        return res

sol = Solution()
print(sol.countMeetings([1,2,3,3,3], [2,2,3,4,4])) #4
print(sol.countMeetings([1,1,2], [1,2,2])) #2
print(sol.countMeetings([1,2,3,3,3], [2,2,5,4,4])) #5
print(sol.countMeetings([1,1,2,2,3,4,5], [3,1,3,2,5,5,5])) #5
print(sol.countMeetings([1,1,1,2,2], [5,5,5,3,3])) #5

4
2
5
5
5


#### Count event days

In [65]:
import heapq
class Solution:
    def maxEvents(self, events: List[List[int]]) -> int:
        start_map = defaultdict(list)
        i, j = float('inf'), 0

        for s, e in events:
            start_map[s].append(e)
            i = min(i, s)
            j = max(j, e)
        
        start_heap = []
        res = 0
        for s in range(i, j + 1):
            while start_heap and start_heap[0] < s:
                heapq.heappop(start_heap)

            for e in start_map[s]:
                heapq.heappush(start_heap, e)

            if start_heap:
                res += 1
                heapq.heappop(start_heap)
        return res
    
    # faster
    def maxEvents_new(self, events: List[List[int]]) -> int:
        events.sort(key=lambda x: (x[1], x[0]))  # Sort events by end day and then by start day
        attendedDays = set()

        for event in events:
            for day in range(event[0], event[1] + 1):
                if day not in attendedDays:
                    attendedDays.add(day)
                    break
        
        return len(attendedDays)


sol = Solution()
print(sol.maxEvents([[1,2],[2,3],[3,4]])) #3
print(sol.maxEvents([[1,5],[1,5],[1,5],[2,3],[2,3]])) #5

3
5


### Kill K-th Bit

In [75]:
def solution(n, k):
    return n & ~(1 << (k - 1))

print(solution(37, 3))


4
33


### Tetris

In [ ]:
class Solution:
    def fillMatrix(self, m: int, n:int, shapes: List[str]) -> List[List[str]]:
        pass

sol = Solution()
print(sol.fillMatrix(3, 6, ['A','B','C','D','E']))

### Tree decrement
1   |       
:-:|
![alt text](Visa_img/tree_1.png "1")  | 

In [ ]:
def getMinCost(val, t_nodes, t_from, t_to):
    global answer
    answer = 0
    n = len(val)
    adj = [[] for i in range(n)]
    for i in range(n-1):
        t_from[i] -=1
        t_to[i] -=1
        adj[t_from[i]].append(t_to[i])
        adj[t_to[i]].append(t_from[i])
    
    def dfs(curr,parent):
        global answer
        count = val[curr] % 2
        for next_ in adj[curr]:
            if next_ == parent:
                continue
            res = dfs(next_,curr)
            answer += res
            count ^= res
        return count
                        
    dfs(0,-1)
    return answer

### Multi dimenstional selection

1   |       
:-:|
![alt text](Visa_img/Multi_selection_1.png "1")  | 

In [ ]:
long long getMaxProduct(vector<vector<int>> arr) { // C++
    int m = int(arr.size()), n = int(arr[0].size()), req = (n + 1) / 2;
    vector<int> A;
    unordered_map<int, vector<int>> mp;
    for (int i = 0; i < m; ++i){
        sort(begin(arr[i]), end(arr[i]));
        for (int j = 0; j < n; ++j){
            A.push_back(arr[i][j]);
            if (mp[arr[i][j]].empty() || mp[arr[i][j]].back() != i){
                mp[arr[i][j]].push_back(i);
            }
        }
    }
    sort(begin(A), end(A));
    A.resize(unique(begin(A), end(A)) - begin(A));
    vector<int> L(m), R(m);
    int cnt = 0, k = 1e9, num = 0;
    unordered_set<int> s;
    for (int i = 0, l = 0; i < int(A.size()); ++i){
        for (int j : mp[A[i]]){
            while(R[j] < n && arr[j][R[j]] == A[i]){
                ++R[j];
                ++cnt;
            }
            if (R[j] - L[j] >= req){
                s.insert(j);
            }
        }
        while(int(s.size()) == m){
            if (A[i] - A[l] < k){
                k = A[i] - A[l];
                num = cnt;
            }else if (A[i] - A[l] == k){
                num = max(num, cnt);
            }
            for (int j : mp[A[l]]){
                while(L[j] < R[j] && arr[j][L[j]] == A[l]){
                    ++L[j];
                    --cnt;
                }
                if (R[j] - L[j] < req){
                    s.erase(j);
                }
            }
            ++l;
        }
    }
    return 1LL * k * num;
}

### Code Signal sample

https://codesignal.com/blog/interview-prep/example-codesignal-questions/

#### Array Manipulation

Given an array a, your task is to output an array b of the same length by applying the following transformation: 
- For each i from 0 to a.length - 1 inclusive, b[i] = a[i - 1] + a[i] + a[i + 1]
- If an element in the sum a[i - 1] + a[i] + a[i + 1] does not exist, use 0 in its place
- For instance, b[0] = 0 + a[0] + a[1]

In [84]:
class Solution:
    def manipulate(self, a: List[int]) -> List[int]:
        a_len, b = len(a), []
        
        for idx in range(a_len):
            val = a[idx]
            if idx > 0:
                val += a[idx - 1]
            if idx < a_len - 1:
                val += a[idx + 1]
            b.append(val)            

        return b

sol = Solution()
print(sol.manipulate([4, 0, 1, -2, 3])) # [4, 5, -1, 2, 1]

[4, 5, -1, 2, 1]


#### String pattern matching

You are given two strings: pattern and source. The first string pattern contains only the symbols 0 and 1, and the second string source contains only lowercase English letters.

Your task is to calculate the number of substrings of source that match pattern. 

We’ll say that a substring source[l..r] matches pattern if the following three conditions are met:
- The pattern and substring are equal in length.
- Where there is a 0 in the pattern, there is a vowel in the substring. 
- Where there is a 1 in the pattern, there is a consonant in the substring. 

Vowels are ‘a‘, ‘e‘, ‘i‘, ‘o‘, ‘u‘, and ‘y‘. All other letters are consonants.

In [90]:
class Solution:
    def match(self, pattern: str, source: str) -> int:
        res = 0

        source = list(source)
        for idx, ch in enumerate(source):
            source[idx] = '0' if ch.lower() in 'aeiouy' else '1'

        pattern = list(pattern)
        pat_len, src_len = len(pattern), len(source)
        for idx in range(src_len - pat_len):
            if pattern == source[idx : idx + pat_len]:
                res += 1

        return res

sol = Solution()
print(sol.match("010", "amazing")) #2
print(sol.match("100", "codesignal")) #0

2
0


#### Two Dimensional Array Traversal

You are given a matrix of integers field of size height × width representing a game field, and also a matrix of integers figure of size 3 × 3 representing a figure. Both matrices contain only 0s and 1s, where 1 means that the cell is occupied, and 0 means that the cell is free.

You choose a position at the top of the game field where you put the figure and then drop it down. The figure falls down until it either reaches the ground (bottom of the field) or lands on an occupied cell, which blocks it from falling further. After the figure has stopped falling, some of the rows in the field may become fully occupied.

Your task is to find the dropping position such that at least one full row is formed. As a dropping position, you should return the column index of the cell in the game field which matches the top left corner of the figure’s 3 × 3 matrix. If there are multiple dropping positions satisfying the condition, feel free to return any of them. If there are no such dropping positions, return -1.

In [99]:
class Solution:
    def arrayTraversal(self, field: List[List[int]], figure: List[List[int]]) -> int:
        ROWS, COLS = len(field), len(field[0])
        figure_len = len(figure)

        for column in range(COLS - figure_len + 1):
            row = 1
            while row < ROWS - figure_len + 1:
                can_fit = True
                for dx in range(figure_len):
                    for dy in range(figure_len):
                        if field[row + dx][column + dy] == 1 and figure[dx][dy] == 1:
                            can_fit = False
                if not can_fit:
                    break
                row += 1
            row -= 1
        
            for dx in range(figure_len):
                row_filled = True
                for column_index in range(COLS):
                    if not (field[row + dx][column_index] == 1 or
                            (column <= column_index < column + figure_len and
                        figure[dx][column_index - column] == 1)):
                        row_filled = False
                if row_filled:
                    return column
        return -1
        

sol = Solution()
print(sol.arrayTraversal(
    [[0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
    [1, 0, 0],
    [1, 1, 0]], 

    [[0, 0, 1],
    [0, 1, 1],
    [0, 0, 1]]
)) # 0
print(sol.arrayTraversal(
    [[0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [1, 1, 0, 1, 0],
    [1, 0, 1, 0, 1]], 

    [[1, 1, 1],
    [1, 0, 1],
    [1, 0, 1]]
)) # 2
print(sol.arrayTraversal(
    [[0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [1, 0, 0, 1],
    [1, 1, 0, 1]],

    [[1, 1, 0],
    [1, 0, 0],
    [1, 0, 0]]
)) # -1

0
2
-1


#### Lookup Table

Given an array of unique integers numbers, your task is to find the number of pairs of indices (i, j) such that i ≤ j and the sum numbers[i] + numbers[j] is equal to some power of 2.

Note: The numbers 20  = 1, 21 = 2, 22 = 4, 23 = 8, etc. are considered to be powers of 2.

In [105]:
class Solution:
    def lookup(self, numbers: List[int]) -> int:
        cnt_map = defaultdict(int)
        max_num = max(numbers)  
        res = 0
        for num in numbers:
            cnt_map[num] += 1

            for power in range(max_num + 1):
                comp_num = (2 ** power) - num
                res += cnt_map[comp_num]
        return res 

sol = Solution()
print(sol.lookup([1, -1, 2, 3])) #5
print(sol.lookup([-2, -1, 0, 1, 2])) #5

5
5


### Lasso

#### Tic Tac Toe

Given n*n grid, determine win, draw, Ongoing

#### Transcations

In [116]:
from datetime import datetime, timedelta
import collections

class Solution:
    def process(self, balances: List[int], requests: list[str]) -> List[int]:
        users_len = len(balances)
        cashbacks = collections.deque([])

        for req_no, req in enumerate(requests):
            typ, tm, hol, amt = req.split(' ')
            
            tm = int(tm)
            hol = int(hol)
            amt = int(amt)

            while cashbacks and tm >= cashbacks[0][0]:
                balances[cashbacks[0][1] - 1] += cashbacks[0][2]
                cashbacks.popleft()

            if not 1 <= hol <= users_len:
                return [-1 * req_no]
            
            if typ == 'deposit':
                balances[hol - 1] += amt
            else:
                if balances[hol - 1] < amt:
                    return [-1 * req_no]
                
                balances[hol - 1] -= amt

                #handle cashback
                current_tm = datetime.fromtimestamp(tm)
                cashback_tm = current_tm + timedelta(hours=24)
                cashbacks.append([cashback_tm.timestamp(), hol, math.floor(0.02*amt)])

        return balances

sol = Solution()
print(sol.process([1000, 1500],
    [
        'withdraw 1613327630 2 480',
        'withdraw 1613327644 2 800',
        'withdraw 1614105244 1 100',
        'deposit 1614108844 2 200',
        'withdraw 1614108845 2 150',
    ]
))


[900, 295]


In [113]:
from datetime import datetime, timedelta

print(datetime.fromtimestamp(1613327630))
print(datetime.fromtimestamp(1613327630) + timedelta(hours=24))
print((datetime.fromtimestamp(1613327630) + timedelta(hours=24)).timestamp())
print(1613327630 + 24 *60 *60)

2021-02-14 13:33:50
2021-02-15 13:33:50
1613414030.0
1613414030


In [146]:
def solve(arr):
    ans = [0] * len(arr)
    mx = 0
    M = 10 ** 9 + 7
    tot = 0
    ttot = 0
    for i in range(len(arr)):
        tot += arr[i]
        tot %= M
        ttot += tot
        ttot %= M
        mx = max(mx, arr[i])
        ans[i] = ttot + mx * (i + 1) % M
        ans[i] %= M
    return ans
    
    
print(solve([1,2,3]))

[2, 8, 19]
